In [45]:
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv("movies.csv", encoding='latin-1')
data.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,Top Gun,PG,1986-05-16,110,6.9,Tom Cruise,236909,Jim Cash,1986
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,Aliens,R,1986-07-18,137,8.4,Sigourney Weaver,540152,James Cameron,1986
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,Flight of the Navigator,PG,1986-08-01,90,6.9,Joey Cramer,36636,Mark H. Baker,1986


In [3]:
#Data understanding
for column in data.columns:
    print(data[column].value_counts())

0.0            2182
20000000.0      207
30000000.0      191
25000000.0      177
15000000.0      170
10000000.0      155
40000000.0      150
35000000.0      149
50000000.0      115
5000000.0       113
60000000.0      107
12000000.0      102
6000000.0        90
8000000.0        88
18000000.0       88
13000000.0       76
7000000.0        73
3000000.0        73
45000000.0       72
17000000.0       65
80000000.0       65
70000000.0       65
11000000.0       62
16000000.0       60
14000000.0       58
22000000.0       57
4000000.0        55
75000000.0       54
55000000.0       54
100000000.0      49
               ... 
260000000.0       1
8900000.0         1
12800000.0        1
10920000.0        1
63600000.0        1
9200000.0         1
65000.0           1
126000000.0       1
29750000.0        1
17700000.0        1
176000000.0       1
169000.0          1
26350000.0        1
5600000.0         1
230000000.0       1
144000000.0       1
11800000.0        1
950000.0          1
2309723.0         1


# Prepearing the dataset for processing

In [4]:
#Get the names of movies with no budget and format the string for a URL
noBudgetMovies = data[data["budget"]==0][["year","name"]]
noBudgetMovies["name"] = noBudgetMovies["name"].str.replace(" ","_")
noBudgetMovies.head()

,year,name
25,1986,Short_Circuit
26,1986,The_Name_of_the_Rose
27,1986,Iron_Eagle
32,1986,Betty_Blue
35,1986,The_Karate_Kid_Part_II


### Requesting the missing budget values from Wikipedia API

In [1]:
from bs4 import BeautifulSoup
import requests

def getPage(movieName):
    S = requests.Session()
    URL = "https://en.wikipedia.org/w/api.php"
    PARAMS = {
        "action": "parse",
        "page": movieName,
        "format": "json"
    }
    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    return DATA

#Getting the budget value
def getBudget(DATA):
    try:
        soup = BeautifulSoup("<html>"+str(DATA["parse"]["text"]["*"])+"</html>",features="html.parser")
    except:
        return "parseError"
    table = soup.find("table",class_="infobox vevent")

    try:
        thList = table.find_all("th")
        tdList = table.find_all("td")
    except:
        return "redirect"

    budgetIndex = -1
    i=0
    for row in thList:
        if "Budget" in str(row):
            budgetIndex = i
            break
        i += 1

    if budgetIndex == -1:
        return "noBudget"
    
    budget = tdList[budgetIndex].contents
    
    try:
        if len(budget)==1 or len(budget)==2 and "reference" in budget[1]["class"]:
            return tdList[budgetIndex].contents[0]
    except:
        pass
    return budget

In [ ]:
#Sends GET requests to wikipedia API to get the missing budget values
#Takes a long time ~35min
dict = {}

for movieName in noBudgetMovies["name"]:
    pageData = getPage(movieName)
    budget = getBudget(pageData)
    
    if budget=="redirect":
        name = movieName+"_(film)"
        pageData = getPage(name)
        budget = getBudget(pageData)
        if budget=="redirect":
            year = noBudgetMovies[noBudgetMovies["name"]==movieName]["year"].values[0]
            name = movieName+"_("+str(year)+"_film)"
            pageData = getPage(name)
            budget = getBudget(pageData)
            if budget=="redirect":
                dict[movieName] = "redirect. last tried name: "+name
                continue
    if budget=="noBudget":
        dict[movieName] = "noBudget"
        continue
    if budget=="parseError":
        dict[movieName] = "parseError"
        continue
    dict[movieName] = budget

In [ ]:
#DO NOT RUN AGAIN IF YOU ALREADY HAVE RESULTS IN A FILE OR MAKE A BACKUP FIRST, BECAUSE API REQUESTS TAKE A LONG TIME TO GET!!!
#Writing dict to file for further processing. 
with open("wikiResults.txt","w",encoding="utf-8") as file:
    for key, value in dict.items():
        file.write(key+"==="+str(value).replace("\n"," ")+"\n")

### Take the budget values and parse them into floats

In [5]:
#Read the results from the file onto a new dictionary
dictFile = {}
with open("wikiResults.txt","r",encoding="utf-8") as file:
    lines = file.readlines()
for line in lines:
    if line=="":
        continue
    split = line.split("===")
    dictFile[split[0]] = split[1].replace("\n","")

In [6]:
def findNumericDigits(string):
    index = 0
    i =0
    for char in string:
        if char.isnumeric():
            index = i
            break
        i += 1
    return index

def parseBudgetValue(budget):
    currencies = {
        "£": 1.33,
        "€": 1.11,
        "AU$": 0.688
    }
    
    try:
        if "sup" in budget:
            return budget
        if "million" in budget or "Million" in budget:
            row = budget.split(" ")
            firstNr = findNumericDigits(row[0])
            value = budget[:firstNr]+str(float(row[0][firstNr:])*1000000)
        elif ","in budget:
            value = budget.replace(",","")
        else:
            value = budget

        if value[0] == "$":
            return float(value[1:])
        elif value[:3] == "US$":
            return float(value[3:])
        elif value[0] == "£":
            return float(value[1:])*currencies["£"]
        elif value[0] == "€":
            return float(value[1:])*currencies["€"]
        elif value[:3] == "AU$":
            return float(value[3:])*currencies["AU$"]
        elif value[:2] == "A$":
            return float(value[2:])*currencies["AU$"]
        else:
            return budget
    except:
        return budget

In [13]:
#Parse the value into a float
for key, value in dictFile.items():
    dictFile[key] = parseBudgetValue(value)

In [12]:
#Manual fixing of remaining errors
errors = ["noBudget","parseError","redirect"]
moviesWithErrors = []
for key, value in dictFile.items():
    if type(value)!= float and value not in errors:
        moviesWithErrors.append(key)
        print(key,value)
        budget = input("Enter new budget value: ")
        if budget == "":
            continue
        dictFile[key] = budget

Summer redirect. last tried name: Summer_(1986_film)
Enter new budget value: 
Link Unknown
Enter new budget value: 
Kangaroo redirect. last tried name: Kangaroo_(1986_film)
Enter new budget value: 
Wings_of_Desire ['5 million ', <a href="/wiki/Deutsche_Mark" title="Deutsche Mark">DM</a>, <sup class="reference" id="cite_ref-FOOTNOTELüdiLüdi200060_3-0"><a href="#cite_note-FOOTNOTELüdiLüdi200060-3">[3]</a></sup>]
Enter new budget value: 5 million Deutsche_Mark
Who's_That_Girl $17–20 million ($37.49 million in 2018)
Enter new budget value: $18.5 million
Wanted:_Dead_or_Alive $4,500,000 (US)
Enter new budget value: $4,500,000
Wisdom ['$6.5 million', <sup class="reference" id="cite_ref-2"><a href="#cite_note-2">[2]</a></sup>, <sup class="reference" id="cite_ref-globe_3-0"><a href="#cite_note-globe-3">[3]</a></sup>]
Enter new budget value: $6.5 million
The_Allnighter approx $1 million
Enter new budget value: $1 million
The_Bedroom_Window ['$8.3 million', <sup class="reference" id="cite_ref-cu

Princess_Mononoke Japan
Enter new budget value: $23.5 million
Perfect_Blue [<span style="white-space: nowrap">¥90 million</span>, <sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup>, ' (', <span style="white-space: nowrap">$6,875,200</span>, ')', <sup class="reference" id="cite_ref-2"><a href="#cite_note-2">[2]</a></sup>]
Enter new budget value: $6,875,200
Open_Your_Eyes ESP 370 million
Enter new budget value: 
The_Castle redirect. last tried name: The_Castle_(1997_film)
Enter new budget value: 
Fireworks redirect. last tried name: Fireworks_(1997_film)
Enter new budget value: $2.3 million
Murder_at_1600 $40-50 million
Enter new budget value: $45 million
Lock,_Stock_and_Two_Smoking_Barrels <div class="plainlist"> <ul><li><a href="/wiki/Pound_sterling" title="Pound sterling">£</a>800,000</li> <li>(<a href="/wiki/United_States_dollar" title="United States dollar">$</a>1.35 million)</li></ul> </div>
Enter new budget value: $1.35 million
Show_Me_Love [<a href="/wiki

Enter new budget value: $27 million
Lady_Vengeance South Korea
Enter new budget value: $4.5 million
London redirect. last tried name: London_(2005_film)
Enter new budget value: 
Idiocracy $2–4 million
Enter new budget value: $3 million
The_Host South Korea
Enter new budget value: US$11 million
Tenacious_D_in_The_Pick_of_Destiny ['$19-20 million', <sup class="reference" id="cite_ref-nyt_1-0"><a href="#cite_note-nyt-1">[1]</a></sup>, <sup class="reference" id="cite_ref-mojo_2-0"><a href="#cite_note-mojo-2">[2]</a></sup>]
Enter new budget value: $19.5 million
Rang_De_Basanti [<span style="white-space: nowrap">₹</span>, '280', <span class="nowrap"> </span>, 'million', <sup class="reference" id="cite_ref-boxofficeindia.com_1-0"><a href="#cite_note-boxofficeindia.com-1">[1]</a></sup>]
Enter new budget value: 
The_Wind_That_Shakes_the_Barley ['€6.5 million', <sup class="reference" id="cite_ref-element_2-4"><a href="#cite_note-element-2">[2]</a></sup>, <sup class="reference" id="cite_ref-numbe

Enter new budget value: 
A_Hijacking ['DKK 15 million', <sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup>, ' (approx. $2.6 million)']
Enter new budget value: $2.6 million
Short_Term_12 under $1 million
Enter new budget value: $1 million
Europa_Report Less than $10 million
Enter new budget value: $10 million
Austenland [<a href="/wiki/Pound_sterling" title="Pound sterling">£</a>, '4.9 million', <br/>, '(', <a href="/wiki/United_States_dollar" title="United States dollar">$</a>, '7.6 million)']
Enter new budget value: $7.6 million
In_a_World... Less than $1 million
Enter new budget value: $1 million
The_Tale_of_the_Princess_Kaguya Japan
Enter new budget value: $49 million
Jimi:_All_Is_by_My_Side ['$5 million', <sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup>, <sup class="reference" id="cite_ref-TN_2-0"><a href="#cite_note-TN-2">[2]</a></sup>]
Enter new budget value: $5 million
The_Congress ['€8 million', <sup class="noprint Inline-Temp

In [20]:
#DO NOT RUN AGAIN. THIS HAS MANUALLY INSERTED VALUES IN IT
#Backup the dictFile with the new manually inserted values
with open("manuallyInserted.txt","w",encoding="utf-8") as file:
    for key, value in dictFile.items():
        file.write(key+"==="+str(value).replace("\n"," ")+"\n")

### Add the newfound values into the set

In [46]:
for key, value in dictFile.items():
    if type(value)==float:
        data[data["name"] == key.replace("_"," ")] = data[data["name"] == key.replace("_"," ")].replace(to_replace=0.0,value=value)

### Encoding labels

In [75]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

##strings to numbers
labelEncoder.fit(data['company'])
data['company'] = labelEncoder.transform(data['company'])

labelEncoder.fit(data['country'])
data['country'] = labelEncoder.transform(data['country'])

labelEncoder.fit(data['director'])
data['director'] = labelEncoder.transform(data['director'])

labelEncoder.fit(data['genre'])
data['genre'] = labelEncoder.transform(data['genre'])

labelEncoder.fit(data['name'])
data['name'] = labelEncoder.transform(data['name'])

labelEncoder.fit(data['rating'])
data['rating'] = labelEncoder.transform(data['rating'])

labelEncoder.fit(data['star'])
data['star'] = labelEncoder.transform(data['star'])

labelEncoder.fit(data['writer'])
data['writer'] = labelEncoder.transform(data['writer'])

labelEncoder.fit(data['released'])
data['released'] = labelEncoder.transform(data['released'])


data.head(20)

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,665,54,2192,1,52287414.0,4661,8,39,89,8.1,2454,299174,3716,1986
1,6000000.0,1680,54,1296,4,70136369.0,1821,7,27,103,7.8,1605,264740,1970,1986
2,15000000.0,1680,54,2645,0,179800601.0,6203,6,23,110,6.9,2345,236909,1852,1986
3,18500000.0,2062,54,1066,0,85160248.0,295,8,33,137,8.4,2192,540152,1629,1986
4,9000000.0,2122,54,2123,1,18564613.0,1872,6,36,90,6.9,1139,36636,2549,1986
5,6000000.0,1158,53,1948,6,138530565.0,3903,8,75,120,8.1,369,317585,2988,1986
6,25000000.0,1160,53,1207,1,12729917.0,2883,6,30,101,7.4,528,102879,977,1986
7,6000000.0,768,54,588,6,8551228.0,771,8,51,120,7.8,924,146768,897,1986
8,9000000.0,1680,54,1004,4,40471663.0,3964,7,9,96,6.8,1730,60565,1970,1986
9,15000000.0,1854,54,557,6,40456565.0,5306,8,38,96,7.5,1039,129698,1333,1986


# Working with the data

In [6]:
labels = data["name"]
movie_data = data.drop(["gross"],1)

X_train, X_test, y_train, y_test = train_test_split(movie_data, labels, test_size = 0.3, random_state = 0)

#predicting
rf = RandomForestClassifier(n_estimators = 200).fit(X_train, y_train)
predictions = rf.predict(X_test)

predictions.tolist()

[4316,
 716,
 3166,
 342,
 6417,
 4325,
 1241,
 2622,
 809,
 1953,
 819,
 2658,
 2407,
 4555,
 1984,
 2582,
 5615,
 5289,
 2832,
 5797,
 5426,
 2784,
 390,
 6255,
 1012,
 1224,
 3718,
 1135,
 1627,
 1890,
 5203,
 215,
 3031,
 3319,
 3495,
 3956,
 1467,
 3583,
 3145,
 5728,
 4962,
 2389,
 4816,
 3401,
 2431,
 5249,
 2637,
 2184,
 1657,
 871,
 5092,
 6383,
 3082,
 405,
 6593,
 1625,
 2781,
 5722,
 3112,
 851,
 3838,
 1627,
 5536,
 740,
 3080,
 1862,
 6499,
 3733,
 6661,
 1264,
 6390,
 6290,
 5648,
 5558,
 809,
 5448,
 342,
 5642,
 4298,
 5524,
 1625,
 797,
 2256,
 2706,
 5536,
 3242,
 5523,
 2450,
 4875,
 1689,
 1533,
 2887,
 1129,
 6083,
 1577,
 6180,
 1613,
 6515,
 4731,
 6639,
 3844,
 5699,
 4456,
 664,
 5358,
 4634,
 2036,
 1618,
 3670,
 5527,
 1728,
 6461,
 1792,
 3249,
 4432,
 6692,
 3892,
 5472,
 3827,
 3851,
 383,
 1657,
 1162,
 4228,
 4957,
 6323,
 4985,
 3465,
 2203,
 5947,
 201,
 1594,
 4204,
 264,
 3431,
 5650,
 4413,
 4110,
 3208,
 53,
 3079,
 3150,
 2674,
 2002,
 3934,
 178

In [7]:
X_test["gross"] = predictions

C:\Users\Silver\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
